<a href="https://colab.research.google.com/github/peisuke/MagicAnimateHandson/blob/main/01_pose_estimation_for_magic_animate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyyaml==5.1

import sys, os, distutils.core

# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!rm -rf detectron2
!git clone --recursive https://github.com/facebookresearch/detectron2
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
# sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 5.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15285, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15285 (delta 2), reused 5 (delta 0), pack-reused 15275
Receiving objects: 100% (15285/15285), 6.18 MiB | 22.44 MiB/s, done.
Resolving deltas: 100% (11116/11116), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.3 MB/s eta 0:00:00
  Pre

In [ ]:
%cd /content/detectron2/projects/DensePose
!mv setup.py setup.py.ORIG
!wget https://www.kkaneko.jp/pro/ni/setup.py
!python setup.py build
!python setup.py install

/content/detectron2/projects/DensePose
--2023-12-05 16:08:13--  https://www.kkaneko.jp/pro/ni/setup.py
Resolving www.kkaneko.jp (www.kkaneko.jp)... 163.145.137.166
Connecting to www.kkaneko.jp (www.kkaneko.jp)|163.145.137.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.kkaneko.jp/ai/ni/setup.py [following]
--2023-12-05 16:08:14--  https://www.kkaneko.jp/ai/ni/setup.py
Reusing existing connection to www.kkaneko.jp:443.
HTTP request sent, awaiting response... 200 OK
Length: 845 [text/x-python]
Saving to: ‘setup.py’

setup.py            100%[===================>]     845  --.-KB/s    in 0s      

2023-12-05 16:08:14 (456 MB/s) - ‘setup.py’ saved [845/845]

running build
running build_py
creating build
creating build/lib
creating build/lib/densepose
copying densepose/config.py -> build/lib/densepose
copying densepose/__init__.py -> build/lib/densepose
creating build/lib/densepose/evaluation
copying densepose/evaluation/densepose_co

In [ ]:
!pip install av

In [ ]:
%cd /content/detectron2/projects/DensePose

/content/detectron2/projects/DensePose


In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!curl -O https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/165712039/model_final_162be9.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  243M  100  243M    0     0  94.9M      0  0:00:02  0:00:02 --:--:-- 94.9M


In [ ]:
!pip install av

In [ ]:
from apply_net import DumpAction
import cv2
import torch
from detectron2.engine.defaults import DefaultPredictor
from detectron2.data.detection_utils import read_image
from apply_net import create_argument_parser

In [ ]:
from densepose.vis.extractor import (
    DensePoseOutputsExtractor,
    DensePoseResultExtractor,
)
from densepose.structures import (
    DensePoseChartPredictorOutput,
    DensePoseEmbeddingPredictorOutput
)
import cv2
import numpy as np
from densepose.structures import DensePoseDataRelative
from densepose.vis.base import Boxes, Image, MatrixVisualizer

In [ ]:
parser = create_argument_parser()
args = parser.parse_args(args=[
    "dump",
    "configs/densepose_rcnn_R_50_FPN_s1x.yaml",
    "model_final_162be9.pkl",
    "input.jpg",
    "--output",
    "dump.pkl"
])

In [ ]:
opts = []
cfg = DumpAction.setup_config(args.cfg, args.model, args, opts)

predictor = DefaultPredictor(cfg)
context = DumpAction.create_context(args, cfg)

count = 0
cap = cv2.VideoCapture("/content/drive/MyDrive/video.mp4")

img_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
img_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

img_size = 512
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter("mask.mp4", fourcc, 25.0, (img_size, img_size))

def predict(img, predictor):
    with torch.no_grad():
        outputs = predictor(img)["instances"]
        outputs = outputs.to("cpu")
    return outputs

val_scale = 255.0 / DensePoseDataRelative.N_PART_LABELS

mask_visualizer = MatrixVisualizer(
    inplace=True, cmap=cv2.COLORMAP_VIRIDIS, val_scale=val_scale, alpha=1.0
)

while True:
    print(count)
    ret, img = cap.read()
    if not ret:
        break
    # 推論してCPUに送る
    outputs = predict(img, predictor)
    outputs = outputs.to("cpu")

    # 推論結果をExtractする
    extractor = DensePoseResultExtractor()
    data = extractor(outputs)
    densepose_result, boxes_xywh = data

    # 姿勢データのレンダリング先、色はcv2.COLORMAP_VIRIDISを使うっぽい
    matrix_scaled_8u = np.zeros((img_height, img_width), dtype=np.uint8)
    matrix_vis = cv2.applyColorMap(matrix_scaled_8u, cv2.COLORMAP_VIRIDIS)

    # 面倒なので最初の一人の姿勢しか使わないよ
    # スコアが高い人を抽出するとか適当にやってね
    for i, result in enumerate(densepose_result):
        # この辺はDensePoseのコピペ
        iuv_array = torch.cat(
            (result.labels[None].type(torch.float32), result.uv * 255.0)
        ).cpu().type(torch.uint8).cpu().numpy()

        bbox_xywh = boxes_xywh.cpu().numpy()[0]
        def _extract_i_from_iuvarr(iuv_arr):
            return iuv_arr[0, :, :]

        matrix = _extract_i_from_iuvarr(iuv_array)
        segm = _extract_i_from_iuvarr(iuv_array)
        mask = np.zeros(matrix.shape, dtype=np.uint8)
        mask[segm >= 0] = 1

        mask_visualizer.visualize(matrix_vis, mask, matrix, bbox_xywh)

        # 二人目以降はbreakで抜けてカット
        break

    # 出力画像は縦横が同じサイズが望ましいのでpadding
    height, width = matrix_vis.shape[:2]
    if height > width:
        pad = (height - width) // 2
        matrix_vis = np.pad(matrix_vis, ((0, 0), (pad, pad), (0, 0)), 'edge')
    elif width > height:
        pad = (width - height) // 2
        matrix_vis = np.pad(matrix_vis, ((pad, pad), (0, 0), (0, 0)), 'edge')

    # maskは24値なので、resizeするときはINTER_NEARESTが望ましい
    matrix_vis = cv2.resize(matrix_vis, (img_size, img_size), cv2.INTER_NEAREST)
    writer.write(matrix_vis)

    count += 1

writer.release()
cap.release()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196


In [ ]:
!cp mask.mp4 "/content/drive/MyDrive/mask.mp4"